In [1]:
import sys
sys.path.append('../')

import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from mtcnn_torch import MTCNN_Torch
from faces_viewport import MTCNN_tf, ViewportsFaceDetector

C:\Users\paulo\anaconda3\envs\faces\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [2]:
dataset360 = pd.read_pickle('../dataframes/dataset360.pkl')
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-35,78,0.994500,"[[(320, 114), (374, 112), (427, 116), (328, 15...","[[(320, 114), (374, 112), (427, 116), (415, 15..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,22,-142,0.523074,"[[(1138, 357), (1152, 357), (1166, 356), (1138...","[[(1138, 357), (1152, 357), (1166, 356), (1167..."
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,46,-103,0.695416,"[[(941, 453), (965, 457), (990, 459), (932, 47...","[[(941, 453), (965, 457), (990, 459), (988, 48..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-39,-95,0.616609,"[[(940, 144), (975, 143), (1010, 144), (945, 1...","[[(940, 144), (975, 143), (1010, 144), (1006, ..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,41,132,0.556615,"[[(149, 430), (170, 431), (190, 431), (147, 45...","[[(149, 430), (170, 431), (190, 431), (192, 45..."


In [3]:
mtcnn_torch = MTCNN_Torch()
#viewports_detector_tf = ViewportsFaceDetector(width = 200)
viewports_detector_torch = ViewportsFaceDetector(torch = True, width = 200)
#mtcnn_tf = MTCNN_tf(verbose = 1)

In [4]:
detections_mtcnn = dataset360.image_path.progress_apply(lambda x: mtcnn_torch.detect_faces_polys(x)[0])

In [5]:
dataset360['detections_mtcnn'] = detections_mtcnn

In [6]:
dataset360.to_pickle('../dataframes/dataset360_detections.pkl')

In [7]:
dataset360.head()

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detections_mtcnn
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-35,78,0.994500,"[[(320, 114), (374, 112), (427, 116), (328, 15...","[[(320, 114), (374, 112), (427, 116), (415, 15...","[[(332, 119), (402, 119), (402, 202), (332, 20..."
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,22,-142,0.523074,"[[(1138, 357), (1152, 357), (1166, 356), (1138...","[[(1138, 357), (1152, 357), (1166, 356), (1167...","[[(1140, 362), (1164, 362), (1164, 395), (1140..."
2,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,46,-103,0.695416,"[[(941, 453), (965, 457), (990, 459), (932, 47...","[[(941, 453), (965, 457), (990, 459), (988, 48...","[[(931, 461), (975, 461), (975, 505), (931, 50..."
3,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-39,-95,0.616609,"[[(940, 144), (975, 143), (1010, 144), (945, 1...","[[(940, 144), (975, 143), (1010, 144), (1006, ...","[[(947, 147), (1000, 147), (1000, 205), (947, ..."
4,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,41,132,0.556615,"[[(149, 430), (170, 431), (190, 431), (147, 45...","[[(149, 430), (170, 431), (190, 431), (192, 45...","[[(150, 437), (186, 437), (186, 480), (150, 48..."
